In [3]:
!git clone https://github.com/DariusAf/MesoNet.git

fatal: destination path 'MesoNet' already exists and is not an empty directory.


In [2]:
# Install system dependencies
!apt-get update
!apt-get install -y build-essential cmake
!apt-get install -y libx11-dev libatlas-base-dev


# Upgrade pip, setuptools, and wheel
!pip install --upgrade pip setuptools wheel


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Ign:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,549 kB]
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,420 kB]
Hit:14 https://ppa.la

In [4]:
# Install face_recognition
!pip install face_recognition

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 7.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566165 sha256=7700d1ca365a31386714f7a4692fbddd221c2c53ba572117f8763894bd30a82f
  Stored in directory: /root/.cache/pip/wheels/7a/eb/cf/e9eced74122b679557f597bb7c8e4c739cfcac526db1fd523d
Successfully built face-recognition-models


In [5]:
!pip install --upgrade dlib

In [6]:
import os

# Change to parent directory
os.chdir('..')

# Verify the current working directory
print("Current directory:", os.getcwd())

Current directory: /


In [7]:
pip install tensorflow keras --upgrade


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.8/47.8 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 601.3/601.3 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 61.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 82.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 111.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.7/347.7 kB 21.0 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.2.0
    Uninstalling ml-dtypes-0.2.0:
      Successfully uninstalled ml-dtypes-0.2.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.15.2
    Uninstalling tensorboard-2.15.2:
      Successfully uninstalled tensorboard-2.15.2
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
  Attempting uninstall: tensorflow
  

In [11]:
cd MesoNet

/content/MesoNet


In [12]:
import numpy as np
from classifiers import *
from pipeline import *

from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [13]:
# 1 - Load the model and its pretrained weights
classifier = Meso4()
classifier.load('weights/Meso4_DF.h5')

In [24]:
# 2 - Minimial image generator
# We did use it to read and compute the prediction by batchs on test videos
# but do as you please, the models were trained on 256x256 images in [0,1]^(n*n)

dataGenerator = ImageDataGenerator(rescale=1./255)
generator = dataGenerator.flow_from_directory(
        'test_images',
        target_size=(256, 256),
        batch_size=1,
        class_mode='binary',
        subset='training')

Found 60 images belonging to 2 classes.


In [25]:
# 3 - Predict
X, y = generator.__next__()
print('Predicted :', classifier.predict(X), '\nReal class :', y)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Predicted : [[0.05753862]] 
Real class : [0.]


In [26]:

# 4 - Prediction for a image dataset

classifier.load('weights/Meso4_F2F.h5')

predictions = compute_accuracy(classifier, 'test_images')
print(predictions)
for video_name in predictions:
    print('`{}` video class prediction :'.format(video_name), predictions[video_name][0])


{}


In [27]:
import os
import numpy as np
import cv2
from tensorflow.keras.preprocessing.image import img_to_array
from classifiers import *
from pipeline import *
from sklearn.metrics import recall_score, accuracy_score
import matplotlib.pyplot as plt

In [28]:
# Load the model and its pretrained weights
classifier = Meso4()
classifier.load('weights/Meso4_DF.h5')

In [29]:
# Function to compute predictions for each class
def compute_accuracy(classifier, directory, use_optimization=True):
    predictions = {'real': [], 'fake': []}
    actual_labels = {'real': [], 'fake': []}
    filenames = {'real': [], 'fake': []}  # To store filenames

    for class_folder in os.listdir(directory):
        class_path = os.path.join(directory, class_folder)
        if os.path.isdir(class_path):
            for filename in os.listdir(class_path):
                if filename.endswith('.jpg') or filename.endswith('.png'):
                    img_path = os.path.join(class_path, filename)

                    # Optimize or resize image based on the flag
                    if use_optimization:
                        img = optimize_image(img_path)
                    else:
                        img = resize_image(img_path)

                    # Convert image to array
                    img_array = img_to_array(img)
                    img_array = img_array / 255.0  # Normalize pixel values
                    img_array = np.expand_dims(img_array, axis=0)  # Expand dimensions for batch

                    # Predict using classifier
                    prediction = classifier.predict(img_array)

                    if class_folder == 'real':
                        predictions['real'].append(prediction[0][0])
                        actual_labels['real'].append(1)  # 1 for real
                        filenames['real'].append(filename)  # Store filename
                    elif class_folder == 'df':
                        predictions['fake'].append(prediction[0][0])
                        actual_labels['fake'].append(0)  # 0 for fake
                        filenames['fake'].append(filename)  # Store filename
    return predictions, actual_labels, filenames


In [30]:
# Function to optimize image using OpenCV (resize, blur, histogram equalization, CLAHE, and sharpening)
from skimage import exposure
from PIL import Image, ImageFilter

import cv2
import numpy as np
from skimage import exposure
from PIL import Image, ImageFilter

def optimize_image(img_path):
    # Read the image using OpenCV
    img = cv2.imread(img_path)

    # Resize image to 256x256
    resized_img = cv2.resize(img, (256, 256))

    # Convert to grayscale
    gray_img = cv2.cvtColor(resized_img, cv2.COLOR_BGR2GRAY)

    # Apply Histogram Equalization
    equalized_img = cv2.equalizeHist(gray_img)

    # Normalize the image (scikit-image)
    norm_img = exposure.rescale_intensity(equalized_img, out_range=(0, 255)).astype(np.uint8)

    # Convert to RGB for consistency and apply sharpening (PIL)
    rgb_img = cv2.cvtColor(norm_img, cv2.COLOR_GRAY2RGB)

    # Convert to LAB color space and enhance edges
    lab_img = cv2.cvtColor(np.array(rgb_img), cv2.COLOR_RGB2LAB)
    l_channel, a_channel, b_channel = cv2.split(lab_img)
    l_channel = cv2.equalizeHist(l_channel)  # Equalize the L channel
    lab_img = cv2.merge([l_channel, a_channel, b_channel])
    enhanced_img = cv2.cvtColor(lab_img, cv2.COLOR_LAB2RGB)

    # Convert back to OpenCV format
    optimized_img = np.array(enhanced_img)

    return optimized_img

In [31]:
# Function to resize image without optimization
def resize_image(img_path):
    img = cv2.imread(img_path)

    # Resize image to 256x256
    resized_img = cv2.resize(img, (256, 256))

    return resized_img

In [1]:
import numpy as np
from sklearn.metrics import (
    accuracy_score, recall_score, precision_score, f1_score,
    confusion_matrix, matthews_corrcoef, balanced_accuracy_score
)
import matplotlib.pyplot as plt

# Calculate metrics
def calculate_metrics(predictions, actual_labels):
    metrics = {}
    for class_type in ['real', 'fake']:
        predicted_classes = np.round(predictions[class_type])
        true_classes = np.array(actual_labels[class_type])

        accuracy = accuracy_score(true_classes, predicted_classes)
        recall = recall_score(true_classes, predicted_classes, pos_label=1 if class_type == 'real' else 0)
        precision = precision_score(true_classes, predicted_classes, pos_label=1 if class_type == 'real' else 0)
        f1 = f1_score(true_classes, predicted_classes, pos_label=1 if class_type == 'real' else 0)
        mcc = matthews_corrcoef(true_classes, predicted_classes)
        balanced_acc = balanced_accuracy_score(true_classes, predicted_classes)

        metrics[class_type] = {
            'accuracy': accuracy,
            'recall': recall,
            'precision': precision,
            'f1': f1,
            'mcc': mcc,
            'balanced_accuracy': balanced_acc
        }

    return metrics

# Calculate overall metrics
def calculate_overall_metrics(predictions, actual_labels):
    predicted_classes = np.concatenate([np.round(predictions['real']), np.round(predictions['fake'])])
    true_classes = np.concatenate([np.array(actual_labels['real']), np.array(actual_labels['fake'])])

    accuracy = accuracy_score(true_classes, predicted_classes)
    recall = recall_score(true_classes, predicted_classes, pos_label=1)  # Assuming 'real' is labeled as 1
    precision = precision_score(true_classes, predicted_classes, pos_label=1)  # Assuming 'real' is labeled as 1
    f1 = f1_score(true_classes, predicted_classes, pos_label=1)  # Assuming 'real' is labeled as 1
    mcc = matthews_corrcoef(true_classes, predicted_classes)
    balanced_acc = balanced_accuracy_score(true_classes, predicted_classes)
    conf_matrix = confusion_matrix(true_classes, predicted_classes)

    return accuracy, recall, precision, f1, mcc, balanced_acc, conf_matrix

# Visualize metrics
def visualize_metrics(metrics_opt, metrics_no_opt, overall_metrics_opt, overall_metrics_no_opt):
    categories = ['Real', 'Fake', 'Overall']
    metrics_to_plot = ['accuracy', 'recall', 'precision', 'f1', 'mcc', 'balanced_accuracy']
    metric_titles = {
        'accuracy': 'Accuracy', 'recall': 'Recall', 'precision': 'Precision',
        'f1': 'F1 Score', 'mcc': 'MCC', 'balanced_accuracy': 'Balanced Accuracy'
    }

    x = np.arange(len(categories))
    width = 0.35
    num_metrics = len(metrics_to_plot)

    fig, ax = plt.subplots(num_metrics, 1, figsize=(10, 5 * num_metrics))

    for idx, metric in enumerate(metrics_to_plot):
        values_opt = [
            metrics_opt['real'].get(metric, 0),
            metrics_opt['fake'].get(metric, 0),
            overall_metrics_opt[idx]
        ]
        values_no_opt = [
            metrics_no_opt['real'].get(metric, 0),
            metrics_no_opt['fake'].get(metric, 0),
            overall_metrics_no_opt[idx]
        ]

        ax[idx].bar(x - width/2, values_opt, width, label='Optimized', color='blue')
        ax[idx].bar(x + width/2, values_no_opt, width, label='Non-Optimized', color='orange')
        ax[idx].set_xticks(x)
        ax[idx].set_xticklabels(categories)
        ax[idx].set_ylabel(metric_titles[metric])
        ax[idx].set_ylim(0, 1)
        ax[idx].set_title(f'{metric_titles[metric]} Comparison')
        ax[idx].legend()

        for i in range(len(categories)):
            opt_value = values_opt[i]
            no_opt_value = values_no_opt[i]

            if opt_value is not None:
                ax[idx].text(i - width/2, opt_value + 0.02, f'{opt_value * 100:.2f}%', ha='center', color='black', fontsize=10)
            else:
                ax[idx].text(i - width/2, 0.5, 'N/A', ha='center', color='black', fontsize=10)

            if no_opt_value is not None:
                ax[idx].text(i + width/2, no_opt_value + 0.02, f'{no_opt_value * 100:.2f}%', ha='center', color='black', fontsize=10)
            else:
                ax[idx].text(i + width/2, 0.5, 'N/A', ha='center', color='black', fontsize=10)

    plt.tight_layout()
    plt.show()

# Compute predictions, actual labels, and filenames for optimized images
predictions_opt, actual_labels_opt, filenames_opt = compute_accuracy(classifier, 'test_images', use_optimization=True)
predictions_no_opt, actual_labels_no_opt, filenames_no_opt = compute_accuracy(classifier, 'test_images', use_optimization=False)

metrics_opt = calculate_metrics(predictions_opt, actual_labels_opt)
overall_metrics_opt = calculate_overall_metrics(predictions_opt, actual_labels_opt)

metrics_no_opt = calculate_metrics(predictions_no_opt, actual_labels_no_opt)
overall_metrics_no_opt = calculate_overall_metrics(predictions_no_opt, actual_labels_no_opt)


print("\nOverall Metrics (Optimized):")
print(f"Accuracy: {overall_metrics_opt[0]:.4f}")
print(f"Recall: {overall_metrics_opt[1]:.4f}")
print(f"Precision: {overall_metrics_opt[2]:.4f}")
print(f"F1 Score: {overall_metrics_opt[3]:.4f}")
print(f"MCC: {overall_metrics_opt[4]:.4f}")
print(f"Balanced Accuracy: {overall_metrics_opt[5]:.4f}")
print(f"Confusion Matrix:\n{overall_metrics_opt[6]}")


print("\nOverall Metrics (Non-Optimized):")
print(f"Accuracy: {overall_metrics_no_opt[0]:.4f}")
print(f"Recall: {overall_metrics_no_opt[1]:.4f}")
print(f"Precision: {overall_metrics_no_opt[2]:.4f}")
print(f"F1 Score: {overall_metrics_no_opt[3]:.4f}")
print(f"MCC: {overall_metrics_no_opt[4]:.4f}")
print(f"Balanced Accuracy: {overall_metrics_no_opt[5]:.4f}")
print(f"Confusion Matrix:\n{overall_metrics_no_opt[6]}")

# Visualize the metrics
visualize_metrics(metrics_opt, metrics_no_opt, overall_metrics_opt, overall_metrics_no_opt)


NameError: name 'compute_accuracy' is not defined

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Visualize metrics
def visualize_metrics(metrics_opt, metrics_no_opt, overall_accuracy_opt, overall_recall_opt, overall_accuracy_no_opt, overall_recall_no_opt):
    categories = ['Real', 'Fake', 'Overall']
    accuracies_opt = [metrics_opt['real']['accuracy'], metrics_opt['fake']['accuracy'], overall_accuracy_opt]
    recalls_opt = [metrics_opt['real']['recall'], metrics_opt['fake']['recall'], overall_recall_opt]
    accuracies_no_opt = [metrics_no_opt['real']['accuracy'], metrics_no_opt['fake']['accuracy'], overall_accuracy_no_opt]
    recalls_no_opt = [metrics_no_opt['real']['recall'], metrics_no_opt['fake']['recall'], overall_recall_no_opt]

    x = np.arange(len(categories))
    width = 0.35

    fig, ax = plt.subplots(3, 1, figsize=(10, 15))

    # Accuracy comparison - Bar chart
    ax[0].bar(x - width/2, accuracies_opt, width, label='Optimized', color='blue')
    ax[0].bar(x + width/2, accuracies_no_opt, width, label='Non-Optimized', color='orange')
    ax[0].set_xticks(x)
    ax[0].set_xticklabels(categories)
    ax[0].set_ylabel('Accuracy')
    ax[0].set_ylim(0, 1)
    ax[0].set_title('Accuracy Comparison')
    ax[0].legend()

    for i in range(len(categories)):
        ax[0].text(i - width/2, accuracies_opt[i] / 2, f'{accuracies_opt[i] * 100:.2f}%', ha='center', color='white', fontsize=12)
        ax[0].text(i + width/2, accuracies_no_opt[i] / 2, f'{accuracies_no_opt[i] * 100:.2f}%', ha='center', color='white', fontsize=12)

    # Recall comparison - Bar chart
    ax[1].bar(x - width/2, recalls_opt, width, label='Optimized', color='blue')
    ax[1].bar(x + width/2, recalls_no_opt, width, label='Non-Optimized', color='orange')
    ax[1].set_xticks(x)
    ax[1].set_xticklabels(categories)
    ax[1].set_ylabel('Recall')
    ax[1].set_ylim(0, 1)
    ax[1].set_title('Recall Comparison')
    ax[1].legend()

    for i in range(len(categories)):
        ax[1].text(i - width/2, recalls_opt[i] / 2, f'{recalls_opt[i] * 100:.2f}%', ha='center', color='white', fontsize=12)
        ax[1].text(i + width/2, recalls_no_opt[i] / 2, f'{recalls_no_opt[i] * 100:.2f}%', ha='center', color='white', fontsize=12)

    # Line graphs for accuracy and recall
    ax[2].plot(categories, accuracies_opt, marker='o', label='Accuracy Optimized', color='blue')
    ax[2].plot(categories, accuracies_no_opt, marker='o', label='Accuracy Non-Optimized', color='orange')
    ax[2].plot(categories, recalls_opt, marker='x', label='Recall Optimized', color='green')
    ax[2].plot(categories, recalls_no_opt, marker='x', label='Recall Non-Optimized', color='red')
    ax[2].set_xticks(x)
    ax[2].set_xticklabels(categories)
    ax[2].set_ylabel('Metrics')
    ax[2].set_ylim(0, 1)
    ax[2].set_title('Line Graphs for Accuracy and Recall')
    ax[2].legend()

    plt.tight_layout()
    plt.show()

# Compute predictions, actual labels, and filenames for optimized images
predictions_opt, actual_labels_opt, filenames_opt = compute_accuracy(classifier, 'test_images', use_optimization=True)
metrics_opt = calculate_metrics(predictions_opt, actual_labels_opt)
overall_accuracy_opt, overall_recall_opt = calculate_overall_metrics(predictions_opt, actual_labels_opt)

# Compute predictions, actual labels, and filenames for non-optimized images
predictions_no_opt, actual_labels_no_opt, filenames_no_opt = compute_accuracy(classifier, 'test_images', use_optimization=False)
metrics_no_opt = calculate_metrics(predictions_no_opt, actual_labels_no_opt)
overall_accuracy_no_opt, overall_recall_no_opt = calculate_overall_metrics(predictions_no_opt, actual_labels_no_opt)

# Visualize the metrics
visualize_metrics(metrics_opt, metrics_no_opt, overall_accuracy_opt, overall_recall_opt, overall_accuracy_no_opt, overall_recall_no_opt)


In [ ]:
# Print metrics
print(f"Optimized Accuracy (Real): {metrics_opt['real']['accuracy'] * 100:.2f}%")
print(f"Optimized Accuracy (Fake): {metrics_opt['fake']['accuracy'] * 100:.2f}%")
print(f"Optimized Overall Accuracy: {overall_accuracy_opt * 100:.2f}%")
print(f"Optimized Recall (Real): {metrics_opt['real']['recall'] * 100:.2f}%")
print(f"Optimized Recall (Fake): {metrics_opt['fake']['recall'] * 100:.2f}%")
print(f"Optimized Overall Recall: {overall_recall_opt * 100:.2f}%")

print(f"Non-Optimized Accuracy (Real): {metrics_no_opt['real']['accuracy'] * 100:.2f}%")
print(f"Non-Optimized Accuracy (Fake): {metrics_no_opt['fake']['accuracy'] * 100:.2f}%")
print(f"Non-Optimized Overall Accuracy: {overall_accuracy_no_opt * 100:.2f}%")
print(f"Non-Optimized Recall (Real): {metrics_no_opt['real']['recall'] * 100:.2f}%")
print(f"Non-Optimized Recall (Fake): {metrics_no_opt['fake']['recall'] * 100:.2f}%")
print(f"Non-Optimized Overall Recall: {overall_recall_no_opt * 100:.2f}%")